# 1. Install and Import Dependencies

In [ ]:
!pip install tensorflow tensorflow-gpu opencv-python matplotlib.pyplot

In [44]:
import tensorflow as tf
import os

# 2. Remove unfit images

In [45]:
import cv2
import imghdr

In [46]:
data_dir = 'cats and dogs-train' # define data directory

In [4]:
image_exts = ['jpeg','jpg', 'bmp', 'png'] # required image extensions

In [ ]:
# Loop through each class in the dataset directory
for image_class in os.listdir(data_dir):  
    # Loop through each image in the current class directory
    for image in os.listdir(os.path.join(data_dir, image_class)):  
        # Construct the full file path for the image
        image_path = os.path.join(data_dir, image_class, image)  
        try:
            # Attempt to read the image using OpenCV
            img = cv2.imread(image_path)  
            # Check the image's file extension/type using imghdr
            tip = imghdr.what(image_path)  
            
            # If the image's extension/type is not in the list of valid extensions
            if tip not in image_exts:  
                # Print a message indicating the image is invalid
                print('Image not in ext list {}'.format(image_path))  
                # Remove the invalid image from the directory
                os.remove(image_path)  
        except Exception as e:
            # If an exception occurs (e.g., corrupted image), print the issue
            print('Issue with image {}'.format(image_path))  
            # os.remove(image_path)


# 3. Load Data

In [47]:
import numpy as np
from matplotlib import pyplot as plt

In [48]:
# Load images from the specified directory and its subdirectories
# 'data_dir' is the path to the folder where images are organized in subdirectories by class
data = tf.keras.utils.image_dataset_from_directory(data_dir, batch_size=32)

# The function automatically:
# - Loads all the images from the 'data_dir'
# - Assigns labels based on the subdirectory names (each subdirectory represents a class)
# - Returns a 'tf.data.Dataset' object that can be used to efficiently handle the image data


Found 557 files belonging to 2 classes.


In [49]:
data_iterator = data.as_numpy_iterator() #convert data to numpy iterator

In [50]:
# Retrieve the next batch of data (images and labels) from the dataset using the NumPy iterator
batch = data_iterator.next()

# The batch contains a tuple with two elements:
# - The first element is a batch of images (as NumPy arrays)
# - The second element is the corresponding labels for those images


In [51]:
batch[0].shape

(32, 256, 256, 3)

In [52]:
import gc
gc.collect()


1406

In [53]:
batch[0][8]

array([[[198.77539 , 195.77539 , 204.77539 ],
        [201.14258 , 198.14258 , 207.14258 ],
        [203.73299 , 200.73299 , 207.84236 ],
        ...,
        [135.22461 , 139.22461 , 150.22461 ],
        [134.94034 , 139.08247 , 147.65607 ],
        [134.77539 , 139.77539 , 145.77539 ]],

       [[197.18404 , 194.18404 , 203.18404 ],
        [199.69336 , 196.69336 , 205.69336 ],
        [202.28932 , 199.28932 , 206.3987  ],
        ...,
        [136.      , 140.      , 149.65234 ],
        [134.73438 , 139.36719 , 146.46875 ],
        [134.      , 139.      , 145.      ]],

       [[194.54297 , 191.54297 , 200.54297 ],
        [197.1211  , 194.1211  , 203.1211  ],
        [199.69922 , 196.69922 , 203.8086  ],
        ...,
        [136.2461  , 140.12305 , 149.12305 ],
        [134.98047 , 139.41237 , 146.5918  ],
        [134.2461  , 139.      , 145.12305 ]],

       ...,

       [[ 86.717926,  57.717926,  42.08789 ],
        [ 86.87695 ,  57.876953,  41.876953],
        [ 86.87695 ,  

# 4. Scale Data

In [54]:
data = data.map(lambda x,y: (x/255, y)) # scale data to min 0, max 1

In [55]:
batch = data.as_numpy_iterator().next()
batch

(array([[[[1.        , 1.        , 1.        ],
          [1.        , 1.        , 1.        ],
          [1.        , 1.        , 1.        ],
          ...,
          [1.        , 1.        , 1.        ],
          [1.        , 1.        , 1.        ],
          [1.        , 1.        , 1.        ]],
 
         [[1.        , 1.        , 1.        ],
          [1.        , 1.        , 1.        ],
          [1.        , 1.        , 1.        ],
          ...,
          [1.        , 1.        , 1.        ],
          [1.        , 1.        , 1.        ],
          [1.        , 1.        , 1.        ]],
 
         [[1.        , 1.        , 1.        ],
          [1.        , 1.        , 1.        ],
          [1.        , 1.        , 1.        ],
          ...,
          [1.        , 1.        , 1.        ],
          [1.        , 1.        , 1.        ],
          [1.        , 1.        , 1.        ]],
 
         ...,
 
         [[1.        , 1.        , 1.        ],
          [1.     

# 5. Split Data

In [56]:
len(data)

18

In [57]:
train_size = int(len(data)*.8)
val_size = int(len(data)*.2)+1

In [58]:
train = data.take(train_size)
val = data.skip(train_size).take(val_size)

# 6. Build Deep Learning Model

In [59]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Flatten

In [60]:
model = Sequential()

In [61]:
# Add a 2D Convolutional layer with 16 filters, a 3x3 kernel, stride of 1, and ReLU activation
# - input_shape: (256, 256, 3) specifies the shape of the input image (256x256 pixels with 3 channels for RGB)
model.add(Conv2D(16, (3,3), 1, activation='relu', input_shape=(256,256,3)))

# Add a MaxPooling layer to reduce the spatial dimensions (downsampling)
# - Pooling reduces the size of the feature maps, helping to reduce computation and prevent overfitting
model.add(MaxPooling2D())

# Add another Conv2D layer with 32 filters, 3x3 kernel, stride of 1, and ReLU activation
# - This layer learns more complex features (such as edges and textures) from the downsampled feature maps
model.add(Conv2D(32, (3,3), 1, activation='relu'))

# Add another MaxPooling layer to further reduce the size of the feature maps
model.add(MaxPooling2D())

# Add another Conv2D layer with 16 filters, 3x3 kernel, stride of 1, and ReLU activation
# - This layer extracts more specific patterns from the features learned in the previous layers
model.add(Conv2D(16, (3,3), 1, activation='relu'))

# Add another MaxPooling layer to further reduce the spatial dimensions
model.add(MaxPooling2D())

# Flatten the 2D feature maps into a 1D vector to prepare for fully connected layers
# - This is required before passing the data to the Dense layers
model.add(Flatten())

# Add a Dense (fully connected) layer with 256 units and ReLU activation
# - This layer learns to combine the features extracted by the Conv2D layers
model.add(Dense(256, activation='relu'))

# Add the final output Dense layer with 1 unit and sigmoid activation
# - Sigmoid activation is used for binary classification (output is between 0 and 1)
model.add(Dense(1, activation='sigmoid'))


In [62]:
model.compile('adam', loss=tf.losses.BinaryCrossentropy(), metrics=['accuracy']) # compile model

In [63]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_3 (Conv2D)           (None, 254, 254, 16)      448       
                                                                 
 max_pooling2d_3 (MaxPooling  (None, 127, 127, 16)     0         
 2D)                                                             
                                                                 
 conv2d_4 (Conv2D)           (None, 125, 125, 32)      4640      
                                                                 
 max_pooling2d_4 (MaxPooling  (None, 62, 62, 32)       0         
 2D)                                                             
                                                                 
 conv2d_5 (Conv2D)           (None, 60, 60, 16)        4624      
                                                                 
 max_pooling2d_5 (MaxPooling  (None, 30, 30, 16)      

# 7. Train

In [64]:
#logdir='logs'

In [65]:
#tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=logdir) # create a tensorboard callback to log metrics

In [66]:
hist = model.fit(train, epochs=20, validation_data=val) # fit model to train data

Epoch 1/20
14/14 [==============================] - 42s 3s/step - loss: 0.7030 - accuracy: 0.4978 - val_loss: 0.6681 - val_accuracy: 0.6881
Epoch 2/20
14/14 [==============================] - 16s 1s/step - loss: 0.6443 - accuracy: 0.6362 - val_loss: 0.6362 - val_accuracy: 0.5872
Epoch 3/20
14/14 [==============================] - 17s 1s/step - loss: 0.5581 - accuracy: 0.7254 - val_loss: 0.4799 - val_accuracy: 0.7982
Epoch 4/20
14/14 [==============================] - 20s 1s/step - loss: 0.4549 - accuracy: 0.7857 - val_loss: 0.3568 - val_accuracy: 0.8899
Epoch 5/20
14/14 [==============================] - 21s 1s/step - loss: 0.3674 - accuracy: 0.8214 - val_loss: 0.2706 - val_accuracy: 0.8991
Epoch 6/20
14/14 [==============================] - 21s 1s/step - loss: 0.2736 - accuracy: 0.8839 - val_loss: 0.1910 - val_accuracy: 0.9266
Epoch 7/20
14/14 [==============================] - 20s 1s/step - loss: 0.1983 - accuracy: 0.9330 - val_loss: 0.1850 - val_accuracy: 0.9266
Epoch 8/20
14/14 [==

# 8. Plot Performance

In [ ]:
fig = plt.figure()
plt.plot(hist.history['loss'], color='teal', label='loss')
plt.plot(hist.history['val_loss'], color='orange', label='val_loss')
fig.suptitle('Loss', fontsize=20)
plt.legend(loc="upper left")
plt.show()

In [ ]:
fig = plt.figure()
plt.plot(hist.history['accuracy'], color='teal', label='accuracy')
plt.plot(hist.history['val_accuracy'], color='orange', label='val_accuracy')
fig.suptitle('Accuracy', fontsize=20)
plt.legend(loc="upper left")
plt.show()

# Test data

In [67]:
data_dir_test = 'cats and dogs-test' # define data directory

In [68]:
# Load images from the specified directory and its subdirectories
# 'data_dir' is the path to the folder where images are organized in subdirectories by class
data_test = tf.keras.utils.image_dataset_from_directory(data_dir_test, batch_size=32) 

Found 140 files belonging to 2 classes.


# 9. Evaluate

In [69]:
from tensorflow.keras.metrics import Precision, Recall, BinaryAccuracy

In [70]:
precision = Precision()
recall = Recall()
accuracy = BinaryAccuracy()

In [71]:
for batch in data_test.as_numpy_iterator(): 
    X, y = batch
    yhat = model.predict(X)
    precision.update_state(y, yhat)
    recall.update_state(y, yhat)
    accuracy.update_state(y, yhat)

1/1 [==============================] - 0s 205ms/step


In [72]:
final_precision = precision.result().numpy()
final_recall = recall.result().numpy()
final_accuracy = accuracy.result().numpy()

# Print the accumulated metrics for the entire dataset
print(f'Precision: {final_precision}')
print(f'Recall: {final_recall}')
print(f'Accuracy: {final_accuracy}')

Precision: 0.6515151262283325
Recall: 0.6142857074737549
Accuracy: 0.6428571343421936


# 10. Make test prediction

In [73]:
import cv2

In [76]:
img = cv2.imread('cute_cat.jpg')
#plt.imshow(img)
#plt.show()

In [77]:
resize = tf.image.resize(img, (256,256))
#plt.imshow(resize.numpy().astype(int))
#plt.show()

In [78]:
yhat = model.predict(np.expand_dims(resize/255, 0))
yhat

1/1 [==============================] - 0s 48ms/step


array([[9.113019e-10]], dtype=float32)

In [79]:
if yhat > 0.5: 
    print(f'Predicted class is Dog')
else:
    print(f'Predicted class is Cat')

Predicted class is Cat


# 11. Save the Model

In [36]:
from tensorflow.keras.models import load_model

In [37]:
model.save(os.path.join('models','classifier.h5'))

In [39]:
new_model = load_model('models/classifier.h5')

In [42]:
yhat_new = new_model.predict(np.expand_dims(resize/255, 0))

1/1 [==============================] - 0s 36ms/step


In [43]:
if yhat_new > 0.5: 
    print(f'Predicted class is Dog')
else:
    print(f'Predicted class is Cat')

Predicted class is Dog
